In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
from skimage.filters import gaussian, sobel, scharr
from skimage.segmentation import active_contour
from skimage.feature import canny
from skimage.exposure import histogram, equalize_hist
from scipy import ndimage as ndi
from skimage.color import label2rgb
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [3]:
x = [0, 2, 3]
np.nanmean([2, 1, np.mean([])])

C:\Users\Tchakamau\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Tchakamau\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.5

In [11]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, 
                          fours, fulls, ratios, inc): 
    
   
    frratios, flratios = ratios
    #print(frratios, flratios)
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    
    
    #----------------------filter------------------------- can use actual filter
    if fours < limit and  num_pconnections+num_gconnections ==0:
        print('fours', fours)        
        print("plasballs")
        plasareas = areas(regions, pblobs, 0.8) 
        print("glasballs")
        glasareas = areas(regions, gblobs, 1.2)
        #if len(glasareas)==nb-npb and len(plasareas)==npb:
        garea = np.nanmean(glasareas)
        plarea = np.nanmean(plasareas)        
        frratios.append(plarea/garea)
        print('frratios', frratios)
        fours += 1
    elif  fulls < limit and num_fconnections ==2 and num_blobs >= 4: 
        fulls += 1
        print('fulls', fulls)     
        imtog = np.copy(frame)
        highs = imtog > threshold_minimum(imtog)
        highs= morphology.opening(highs, disk(8))
        imtog[highs==0]=0
        img = scharr(imtog)
        markers = np.zeros_like(frame)
        markers[imtog < threshold_mean(frame)] = 1
        markers[imtog > threshold_minimum(frame)+2] = 2
        segmentation = morphology.watershed(img, markers)
        segmentation = ndi.binary_fill_holes(morphology.closing(segmentation - 1, disk(5)))
        labeled_balls_tog, _ = ndi.label(segmentation)
        region=measure.regionprops(labeled_balls_tog)
        

        allarea = areafinder(regions)
        print('allarea', allarea)
        plarea = areafinder(region)/npb
        print('plarea', plarea)
        glarea = (allarea-npb*plarea)/(nb-npb)
        print('glarea', glarea)
        flratios.append(plarea/glarea)
        print('flratios', flratios)    
            
    else:
        #print(num_fconnections, 'connections', ' ', num_blobs, 'blobs')
        pass
    #print(ratios)
    return [fours, fulls, ratios]



        
# take largest isodata image region, and return aclassification
def areas(regions, blobs, rang):    
    # mathc blob o blob, collect areas into a list
    if len(regions)==0 or len(blobs)==0:
        return([])
    else: 
        areas = []
        
        return(areas)

def areafinder(allregions):    
    # mathc blob o blob, collect areas into a list
    #print(allregions[0].area)
    aregion = max(allregions, key=attrgetter('area'))
    return(aregion.area)
"""def reducer(connections):
    return sum(connections)  """
def within(pt, origin, rad):
    return (vector_dist(pt, origin) <= rad)
def connect(cv, radii, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    num_f_connections = 0
    norm_radius = 15
    total_separations = 0
    normalized_dists = []
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        total_separations += vector_d
        r1,r2 = radii
        if vector_d <= 2*r1 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_connections += 1
        if vector_d <= 2*r2 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_f_connections += 1
        else:
            pass
        if vector_d <= 4*r1: 
            normalized_dists.append(vector_d/(2*norm_radius))
    return (num_connections, num_f_connections)

def reducer(connections):
    if len(connections)== 0:
        return [0, 0]
    else:
        return(np.sum(connections, axis=0)    )

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

In [12]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/W*half/*_21_a.avi")#glob("./tchakamau/W*half/*_[4,7,8]*.avi") +glob("./tchakamau/W*half/*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)

trans_runs = {}
#trans_fil = 10

ave = np.average
nb = 4
npb=2
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
fp_bond = 15*1.75
fg_bond = 15*1.8
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [nb , expected_connections, p_bond, g_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
limit = 5
frratio = []
flratio = []
ratios = [frratio, flratio]
        
for vidnum in range(len(filenames)):
    print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
    #meta = ffprobe(filenames[vidnum])
    #print(meta)#['@nb_frames'])
    #print(5)
    frames = vreader(filenames[vidnum])
    #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    #print(frames)



    sides = {"p":[], "g":[], "b":[], "ucf":[]}
    classes = {"p":[], "g":[], "b":[], "ucf":[]}
    transitions = [("p->p",[]),("p->g",[]), ("p->b",[]),
                   ("g->p",[]),("g->g",[]), ("g->b",[]),
                  ("b->p",[]),("b->g",[]), ("b->b",[])]

    transitions = OrderedDict(transitions)
    broken_count = 0
    class_num = 0
    origin = 'ucf'
    last_class = 'ucf'
    last_whole = 0
    fours = 0
    fulls = 0
    fnum = 0
    inc = 20
    for fr in frames:
        frame = fr[:, :, 2]
        org = total_threshold_filter(frame, fours, fulls, ratios, inc)
        fours, fulls, ratios = org
        #print(fours, fulls)
        if fours >= limit and fulls >= limit:
            break
    print(fours, fulls, ratios)

    index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------



['./tchakamau\\Washed15vhalf\\45782_15Vpp_21_a.avi']
Number files:  1
Processing vid 0 : ./tchakamau\Washed15vhalf\45782_15Vpp_21_a.avi
fulls 1
allarea 2790
plarea 669.5
glarea 725.5
flratios [0.9228118538938663]
fulls 2
allarea 2818
plarea 647.0
glarea 762.0
flratios [0.9228118538938663, 0.8490813648293963]
fulls 3
allarea 2829
plarea 659.5
glarea 755.0
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344]
fulls 4
allarea 3290
plarea 683.5
glarea 961.5
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344, 0.7108684347373895]
fulls 5
allarea 3240
plarea 685.0
glarea 935.0
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344, 0.7108684347373895, 0.732620320855615]
fours 0
plasballs
[ 96. 118.  17.]
[ 80. 156.  17.]
glasballs
[ 58. 144.  13.]
[  0. 120.   9.]
fours 0
plasballs
[123.  80.  17.]
(121.2722029988466, 80.23875432525952)
match 867
[73. 83. 17.]
(239.74271844660194, 228.16504854368932)
glasballs
[52. 92. 15.]
(121.2722029988466, 8

fours 0
plasballs
[ 93. 120.  17.]
(43.0, 111.0)
(239.4626168224299, 229.58878504672896)
[57. 81. 17.]
(43.0, 111.0)
(239.4626168224299, 229.58878504672896)
glasballs
[76. 99. 13.]
(43.0, 111.0)
(239.4626168224299, 229.58878504672896)
[  0. 121.   9.]
(43.0, 111.0)
(239.4626168224299, 229.58878504672896)
fours 0
plasballs
[ 95. 119.  17.]
(240.4814814814815, 229.36574074074073)
[58. 80. 17.]
(240.4814814814815, 229.36574074074073)
glasballs
[77. 98. 13.]
(240.4814814814815, 229.36574074074073)
[  0. 121.   9.]
(240.4814814814815, 229.36574074074073)
fours 0
plasballs
[ 96. 119.  17.]
(41.80392156862745, 110.44444444444444)
[59. 78. 17.]
(41.80392156862745, 110.44444444444444)
glasballs
[78. 97. 13.]
(41.80392156862745, 110.44444444444444)
[  0. 121.   9.]
(41.80392156862745, 110.44444444444444)
fours 0
plasballs
[ 97. 118.  17.]
(239.90575916230367, 230.2198952879581)
[60. 76. 17.]
(239.90575916230367, 230.2198952879581)
glasballs
[78. 95. 13.]
(239.90575916230367, 230.2198952879581)
[

fours 0
plasballs
[ 79. 130.  17.]
[ 34. 147.  17.]
glasballs
[ 64. 151.  13.]
[  0. 118.  16.]
fours 0
plasballs
[ 80. 131.  17.]
(239.9633507853403, 229.64921465968587)
[ 34. 145.  17.]
(239.9633507853403, 229.64921465968587)
glasballs
[ 65. 148.  13.]
(239.9633507853403, 229.64921465968587)
[  0. 117.  16.]
(239.9633507853403, 229.64921465968587)
fours 0
plasballs
[ 80. 131.  17.]
(240.33004926108376, 228.69950738916256)
[ 32. 142.  17.]
(240.33004926108376, 228.69950738916256)
glasballs
[ 65. 146.  13.]
(240.33004926108376, 228.69950738916256)
[  1. 118.  16.]
(240.33004926108376, 228.69950738916256)
fours 0
plasballs
[ 80. 132.  17.]
(35.311688311688314, 247.6883116883117)
[ 32. 141.  17.]
(35.311688311688314, 247.6883116883117)
glasballs
[ 63. 142.  13.]
(35.311688311688314, 247.6883116883117)
[  0. 119.  16.]
(35.311688311688314, 247.6883116883117)
fours 0
plasballs
[ 81. 132.  17.]
(122.0, 131.0)
[ 31. 140.  17.]
(122.0, 131.0)
glasballs
[ 61. 139.  13.]
(122.0, 131.0)
[  0. 11

fours 0
plasballs
[43. 78. 17.]
(240.55223880597015, 228.92537313432837)
[ 1. 90. 17.]
(240.55223880597015, 228.92537313432837)
glasballs
[21. 47. 13.]
(240.55223880597015, 228.92537313432837)
[  0. 120.   9.]
(240.55223880597015, 228.92537313432837)
fours 0
plasballs
[44. 77. 17.]
(16.9, 51.076623376623374)
(43.0, 110.0)
(240.635, 229.415)
[ 0. 92. 17.]
(16.9, 51.076623376623374)
(43.0, 110.0)
(240.635, 229.415)
glasballs
[23. 47. 13.]
(16.9, 51.076623376623374)
match 770
[  0. 120.   9.]
(43.0, 110.0)
(240.635, 229.415)
fours 0
plasballs
[85. 78. 17.]
(72.456, 30.536)
(240.24867724867724, 229.7089947089947)
[ 54.         105.          16.35714286]
(72.456, 30.536)
(240.24867724867724, 229.7089947089947)
glasballs
[82. 97. 14.]
(72.456, 30.536)
(240.24867724867724, 229.7089947089947)
[78. 25. 13.]
(72.456, 30.536)
match 750
[  0. 120.   9.]
(240.24867724867724, 229.7089947089947)
fours 0
plasballs
[84. 77. 17.]
(74.07340720221606, 24.75207756232687)
(240.13761467889907, 228.8486238532